In [1]:
%load_ext autoreload
%autoreload 2

In [140]:
import ee, os
from pathlib import Path
from datetime import datetime
from tqdm import tqdm
import time
import rasterio
import rasterio.warp
from utils import get_project_data

In [4]:
# Project data
gt_file, gt, gt_date, gt_crs, polygon, bounds = get_project_data(
    "498",
    gt_dir="../assets/data/preprocessed",
    gt_data_bands=list(range(1,6)),
    shapefile_paths=['../assets/data/NHM_projectDekning_AOI_edit2015_V2.shp', 
                    '../assets/data/ALS_projects_Dz_all_norway.shp']
)

In [7]:
bounds[0][0]

[(9.703842793092072, 60.7253957927164),
 (10.258106843349504, 60.933251765941094)]

In [5]:
import ee
ee.Initialize(project="ee-walteliot")

In [131]:
import geetools
import ee
from uuid import uuid4
from geetools.utils import makeName
from geetools import tools
from geetools.batch import utils
from hashlib import sha256

def exportImageCollectionToDrive(
    collection, 
    fn_prefix, 
    folder, 
    scale, 
    datatype, 
    region, 
    crs, 
    verbose=False, 
    **kwargs
):
    """
    Slight modification of geetools.batch.Export.imagecollection.toDrive to use a filename prefix 
    to define the name property when exporting images
    """
    # compat
    namePattern='{id}'
    datePattern=None
    extra=None
    dataType=datatype
    # empty tasks list
    tasklist, filenames = [], []
    # get region
    if region:
        region = tools.geometry.getRegion(region)
    # Make a list of images
    img_list = collection.toList(collection.size())
    n = 0
    while True:
        try:
            img = ee.Image(img_list.get(n))
            name = fn_prefix+"{}-{}".format(
                image.id().getInfo().split("_")[0], 
                datetime.now().strftime("%Y%d%mT%H%M%S"))
            description = utils.matchDescription(makeName(img, namePattern, datePattern, extra).getInfo())
            # convert data type
            img = utils.convertDataType(dataType)(img)
            if region is None:
                region = tools.geometry.getRegion(img)
            task = ee.batch.Export.image.toDrive(image=img,
                                                 description=description,
                                                 folder=folder,
                                                 fileNamePrefix=name,
                                                 region=region,
                                                 scale=scale,
                                                 crs=crs)
            task.start()
            if verbose: print(f"Submitted new task: taskId={task.id}, name={name}, description={description}")
            tasklist.append(task)
            filenames.append(name)
            n += 1
        except Exception as e:
            error = str(e).split(':')
            if error[0] == 'List.get': 
                if verbose: print(f"Reached end of image list at index: {n}")
                break
            else: raise e
    return tasklist, filenames

projection = ee.Projection('EPSG:4326')
geometry = ee.Geometry.Rectangle(bounds[0][0], proj=projection)
dates = (ee.Date('2017-07-01'), ee.Date('2017-07-11'))
s2_bands = ['B1','B2','B3','B4','B5','B6','B7','B8','B8A', 'B9', 'B11','B12','MSK_CLDPRB']
icol = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
       .filterBounds(geometry)
       .filterDate(*dates)
       .select(s2_bands))
print("Exporting icol with", icol.size().getInfo(), "images.")
# Geetools export
tasks, filenames = exportImageCollectionToDrive(
    collection=icol,
    region=geometry,
    folder="geeExports",
    scale=10,
    fn_prefix="498-geetoolsWithFn-",
    datatype="uint16",
    crs=projection.getInfo()["crs"],
    verbose=True
)
for t, f in zip(tasks, filenames):
    print(task, "->", f)

Exporting icol with 2 images.
Submitted new task: taskId=NMSE6VT7MIYRSLMVVI7MKFFT, name=498-geetoolsWithFn-20170526T105031-20231605T135247, description=20170702T104021_20170702T104252_T32VNN
Submitted new task: taskId=FJEIFC6GWFAQT47FXC4P3U5N, name=498-geetoolsWithFn-20170526T105031-20231605T135250, description=20170705T105031_20170705T105026_T32VNN
Reached end of image list at index: 2
<Task NAANTYZR6L36FP5QKWY6T6QA EXPORT_IMAGE: 498-test-list-13-reprojected-clipped-20170526T105031 (UNSUBMITTED)> -> 498-geetoolsWithFn-20170526T105031-20231605T135247
<Task NAANTYZR6L36FP5QKWY6T6QA EXPORT_IMAGE: 498-test-list-13-reprojected-clipped-20170526T105031 (UNSUBMITTED)> -> 498-geetoolsWithFn-20170526T105031-20231605T135250


In [133]:
tasks[0].id

'NMSE6VT7MIYRSLMVVI7MKFFT'

In [134]:
import ee
from datetime import datetime

def cast_image(image, datatype):
    try:
        return {
            "float":image.toFloat(), 
            "byte":image.toByte(), 
            "int":image.toInt(),
            "double":image.toDouble(),
            "long": image.toLong(),
            "short": image.toShort(),
            "int8": image.toInt8(),
            "int16": image.toInt16(),
            "int32": image.toInt32(),
            "int64": image.toInt64(),
            "uint8": image.toUint8(),
            "uint16": image.toUint16(),
            "uint32": image.toUint32()
        }[datatype]
    except KeyError as e:
        raise KeyError(f"Invalid datatype")

class GEEDownloader:
    def __init__(
        self,
        gee_project: str,
        crs: str="EPSG:4326",
        collection_name: str="COPERNICUS/S2_SR_HARMONIZED"
    ):
        ee.Initialize(project=gee_project)
        self.projection = ee.Projection(crs)
        self.crs = self.projection.getInfo()["crs"]
        self.crsTrasform = self.projection.getInfo()["transform"]
        self.collection_name = collection_name
    
    def downloadTimeserie(
        self,
        bbox: list, # [[xmin,ymin],[xmax,ymax]]
        fn_prefix: str, # i.e. projectId
        drive_folder: str,
        gt_date: datetime,
        date_offset_amount: int,
        date_offset_unit: str="day",
        date_offset_policy: str="both",
        bands: list=['B1','B2','B3','B4','B5','B6','B7','B8','B8A', 'B9', 'B11','B12','MSK_CLDPRB'],
        datatype: str="uint16"
    ):
        assert date_offset_policy in ["before", "after", "both"], f'Invalid date_offset_policy: {date_offset_policy}. Value must be in {["before", "after", "both"]}.'
        # Configure
        geometry = ee.Geometry.Rectangle(bbox, proj=self.projection)
        if date_offset_policy == "before": start_offset, end_offset = -date_offset_amount, 0
        elif date_offset_policy == "after": start_offset, end_offset = 0, date_offset_amount
        else: start_offset, end_offset = -date_offset_amount, date_offset_amount
        dates = (
            ee.Date(gt_date).advance(start_offset, date_offset_unit), 
            ee.Date(gt_date).advance(end_offset, date_offset_unit)
        )
        # ImageCollection
        icol = (ee.ImageCollection(self.collection_name)
           .filterBounds(geometry)
           .filterDate(*dates)
           .select(s2_bands))
        # Create download tasks
        fn_prefix = fn_prefix+"-GEE-"+self.collection_name.replace("/", "_")+"-"
        tasks, filenames = exportImageCollectionToDrive(
            collection=icol,
            region=geometry,
            folder=drive_folder,
            scale=10,
            fn_prefix=fn_prefix,
            datatype=datatype,
            crs=projection.getInfo()["crs"],
            verbose=True
        )
        return tasks, filenames
    
def download_gee_tasks(drive, tasks, filenames, drive_folder, local_folder):
    folderId = drive.getFolderId(drive_folder)
    while len(tasks)>0:
        for filename, task in zip(filenames, tasks):
            if task.state == "COMPLETED":
                try:
                    print(f"Downloading {drive_folder}/{filename}")
                    fileId = drive.getFileId(folename, folderId)
                    drive.downloadFile(fileId=fileId, localdir=local_folder)
                    try:
                        drive.deleteFile(fileId)
                        tasks.remove(task)
                        filenames.remove(filename)
                    except:
                        print(f"Could not delete file: fileId={fileId}, filename={filename}")
                        print(e)
                except:
                    print(f"Could not download task: taskId={task.id}, filename={filename}")
                    print(e)                            
            elif task.state == "FAILED": 
                tasks.remove(task)
                filenames.remove(task)
                print(f"Task(taskId={task.id}, fileName={filename}) has failed.")
            elif task.state == "CANCELLED": 
                tasks.remove(task)
                filenames.remove(task)
                print(f"Task(taskId={task.id}, fileName={filename}) was cancelled.")
            else:
                print(f"Unkown task state: {task.state}")

# Reprojection

In [1]:
import gdal
import os
from utils import get_project_data
from pathlib import Path

In [2]:
# GT data
gt_path, gt_file, _, gt_date, gt_crs, _, bbox = get_project_data(
    "498",
    gt_dir="../assets/data/preprocessed",
    gt_data_bands=list(range(1,6)),
    shapefile_paths=['../assets/data/NHM_projectDekning_AOI_edit2015_V2.shp', 
                    '../assets/data/ALS_projects_Dz_all_norway.shp']
)

# Reference files
ref_paths = [p.path for p in os.scandir("../assets/data/sentinel_data/s2_reprojected/764") 
             if p.path.endswith(".tif")]
# GEE paths
gee_paths = [p.path for p in os.scandir("../gee_data_dev/764") if p.path.endswith(".tif")]

In [3]:
gt_file.name

'../assets/data/preprocessed/498.tif'

In [8]:
def warp(input_path, ref_path):
    # Set vars
    os.environ['PROJ_LIB'] = '/scratch/ewalt/miniconda3/envs/bfs-3.7/share/proj'
    os.environ['GDAL_DATA'] = '/scratch/ewalt/miniconda3/envs/bfs-3.7/share/'
    # Paths
    input_path = str(input_path)
    output_path = str(input_path).replace(Path(input_path).stem, Path(input_path).stem+"_reprojected")
    ref_path = str(ref_path)
    print(f"Warping {input_path} to {output_path} according to {ref_path}")
    # Load reference dataset
    ref_ds = gdal.Open(ref_path)
    # warp options
    warp_options = gdal.WarpOptions(
        format="GTiff",
        dstSRS=ref_ds.GetProjectionRef(),
        xRes=10.0, yRes=-10.0
    )
    # reproject
    out_ds = gdal.Warp(output_path, input_path, options=warp_options)
    # Unset vars
    os.environ['PROJ_LIB'] = ""
    os.environ['GDAL_DATA'] = ""
    return ref_ds, out_ds

In [9]:
def compute_intersection(ds1, ds2):
    assert ds1.GetProjectionRef() == ds2.GetProjectionRef()
    origin_x1, a1, b1, origin_y1, d1, e1 = ds1.GetGeoTransform()
    origin_x2, a2, b2, origin_y2, d2, e2 = ds2.GetGeoTransform()
    r1 = [
        origin_x1, origin_y1 ,origin_x1 + (a1* ds1.RasterXSize), 
        origin_y1 + (e1 * ds1.RasterYSize)
    ]    
    r2 = [
        origin_x2, origin_y2 ,origin_x2 + (a2* ds2.RasterXSize), 
        origin_y2 + (e2 * ds2.RasterYSize)
    ]
    assert (a1 == a2 and e1 == e2)  
    xmin = max(r1[0], r2[0])
    ymax = min(r1[1], r2[1])
    xmax = min(r1[2], r2[2])
    ymin = max(r1[3], r2[3])
    return (xmin,ymin,xmax,ymax)

def geo_coords_tooffset(ds, lims):
    xmin,ymin,xmax,ymax = lims  
    origin_x, pixel_w, b, origin_y, d, pixel_h = ds.GetGeoTransform()
    xoff = int((xmin - origin_x)/pixel_w)
    yoff = int((origin_y - ymax)/pixel_w)
    xsize = int((xmax - xmin)/pixel_w)
    ysize = int((ymax - ymin)/pixel_w)  
    return (xoff, yoff,xsize,ysize)

In [10]:
ref_ds, out_ds = warp(gee_paths[1], ref_paths[0])

Warping ../gee_data_dev/764/764-GEE_COPERNICUS_S2_SR_HARMONIZED-20170406T105021-20231605T191639.tif to ../gee_data_dev/764/764-GEE_COPERNICUS_S2_SR_HARMONIZED-20170406T105021-20231605T191639_reprojected.tif according to ../assets/data/sentinel_data/s2_reprojected/764/764_S2A_MSIL2A_20170523T104031_N9999_R008_T32VNM_20201207T090240.tif


In [11]:
print(f"ref size: {ref_ds.RasterXSize, ref_ds.RasterYSize}, out size: {out_ds.RasterXSize, out_ds.RasterYSize}")

ref size: (1743, 3845), out size: (1884, 3846)


In [27]:
compute_intersection(ref_ds, out_ds)

(558766.7500601338, 6732407.033205436, 568185.6051443298, 6649259.037182807)

In [15]:
out_dir = os.path.dirname(gee_paths[0])+"_reprojected"

'../gee_data/764_reprojected'

In [16]:
gt_file

NameError: name 'gt_file' is not defined

In [23]:
!which python

/scratch/ewalt/miniconda3/bin/python


In [47]:
ref_paths[0], gee_paths[0]

('../assets/data/sentinel_data/s2_reprojected/764/764_S2A_MSIL2A_20170523T104031_N9999_R008_T32VNM_20201207T090240.tif',
 '../gee_data/764/764-GEE_COPERNICUS_S2_SR_HARMONIZED-20170403T104021-20231605T191636.tif')

In [48]:
os.path.exists("../gee_data/764/764-GEE_COPERNICUS_S2_SR_HARMONIZED-20170403T104021-20231605T191636_reprojected.tif")

True

In [43]:
out_ds.RasterXSize, out_ds.RasterYSize

(1884, 3846)

In [19]:
# Paths
input_path = Path(gee_paths[0])
output_path = Path(str(input_path).replace(input_path.stem, input_path.stem+"_reprojected"))
output_path, gt_path = str(output_path), str(gt_path)
# Read gt
gt_ds = gdal.Open(str(gt_path))
# warp options
warp_options = gdal.WarpOptions(
    format="GTiff",
    dstSRS=gt_ds.GetProjectionRef(),
    xRes=10.0, yRes=-10.0
)
# reproject dataset
reprojected_dataset = gdal.Warp(output_path, input_path, options=warp_options)
gt_proj

ERROR 1: PROJ: pj_obj_create: Open of /scratch/ewalt/miniconda3/envs/bfs-3.7/share/proj failed
ERROR 1: PROJ: createGeodeticReferenceFrame: Open of /scratch/ewalt/miniconda3/envs/bfs-3.7/share/proj failed
ERROR 1: PROJ: proj_as_wkt: Open of /scratch/ewalt/miniconda3/envs/bfs-3.7/share/proj failed
ERROR 1: PROJ: proj_create_from_wkt: Open of /scratch/ewalt/miniconda3/envs/bfs-3.7/share/proj failed
ERROR 1: PROJ: proj_as_wkt: Open of /scratch/ewalt/miniconda3/envs/bfs-3.7/share/proj failed
ERROR 1: PROJ: proj_create_from_wkt: Open of /scratch/ewalt/miniconda3/envs/bfs-3.7/share/proj failed
ERROR 1: PROJ: createGeodeticReferenceFrame: Open of /scratch/ewalt/miniconda3/envs/bfs-3.7/share/proj failed
ERROR 1: PROJ: pj_obj_create: Open of /scratch/ewalt/miniconda3/envs/bfs-3.7/share/proj failed
ERROR 1: PROJ: proj_as_wkt: Open of /scratch/ewalt/miniconda3/envs/bfs-3.7/share/proj failed
ERROR 1: PROJ: proj_as_wkt: Open of /scratch/ewalt/miniconda3/envs/bfs-3.7/share/proj failed
ERROR 1: PROJ:

SystemError: <built-in function wrapper_GDALWarpDestName> returned NULL without setting an error

In [13]:
gdal_reproject(gee_paths[0], gt_path)

ERROR 1: PROJ: pj_obj_create: Open of /scratch/ewalt/miniconda3/envs/bfs-3.7/share/proj failed
ERROR 1: PROJ: createGeodeticReferenceFrame: Open of /scratch/ewalt/miniconda3/envs/bfs-3.7/share/proj failed
ERROR 1: PROJ: proj_as_wkt: Open of /scratch/ewalt/miniconda3/envs/bfs-3.7/share/proj failed
ERROR 1: PROJ: proj_create_from_wkt: Open of /scratch/ewalt/miniconda3/envs/bfs-3.7/share/proj failed
ERROR 1: PROJ: proj_as_wkt: Open of /scratch/ewalt/miniconda3/envs/bfs-3.7/share/proj failed
ERROR 1: PROJ: proj_create_from_wkt: Open of /scratch/ewalt/miniconda3/envs/bfs-3.7/share/proj failed
ERROR 1: PROJ: createGeodeticReferenceFrame: Open of /scratch/ewalt/miniconda3/envs/bfs-3.7/share/proj failed
ERROR 1: PROJ: pj_obj_create: Open of /scratch/ewalt/miniconda3/envs/bfs-3.7/share/proj failed
ERROR 1: PROJ: proj_as_wkt: Open of /scratch/ewalt/miniconda3/envs/bfs-3.7/share/proj failed
ERROR 1: PROJ: proj_as_wkt: Open of /scratch/ewalt/miniconda3/envs/bfs-3.7/share/proj failed
ERROR 1: PROJ:

SystemError: <built-in function wrapper_GDALWarpDestName> returned NULL without setting an error

In [5]:
gt_path

PosixPath('../assets/data/preprocessed/764.tif')

In [8]:
gdal.Open(gt_path)

RuntimeError: not a string

In [12]:
gdal.Open(str(gt_path)).close()

[autoreload of gee_download failed: Traceback (most recent call last):
  File "/scratch/ewalt/miniconda3/envs/bfs-3.7/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/scratch/ewalt/miniconda3/envs/bfs-3.7/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/scratch/ewalt/miniconda3/envs/bfs-3.7/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/scratch/ewalt/miniconda3/envs/bfs-3.7/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line 791, in source_to_code
  File "<frozen importlib._bootstrap>", line 

AttributeError: type object 'object' has no attribute '__getattr__'

In [36]:
!conda activate bfs-3.7


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.


